## I executed this in colab and transferred the trained models to the results section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('drive/MyDrive/Work/multi-omics/')

! pip install mudata
import mudata as md
from mudata import MuData
! pip install anndata
import anndata as ad

import torch
import numpy as np
import pandas as pd

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 13.6 MB/s eta 0:00:00


## preprocess data

### brain

In [ ]:
# get data
data_prefix = './brain/'
mdata = md.read(data_prefix+'mudata.h5mu', backed=False)
# load train_val_test split for fairness
import pandas as pd
split = pd.read_csv(data_prefix+'train_val_test_split.csv')
#train_indices = split[split['is_train'] == 'train']['num_idx'].values
not_test_indices = split[split['is_train'] != 'iid_holdout']['num_idx'].values
#mdata = mdata[not_test_indices,:]
mdata

MuData object with n_obs × n_vars = 3143 × 110849
  obs:	'celltype'
  var:	'name', 'modality'
  2 modalities
    rna:	3143 x 15172
      obs:	'ID', 'rna_celltype', 'atac_celltype'
      var:	'name', 'modality'
    atac:	3143 x 95677
      obs:	'ID', 'rna_celltype', 'atac_celltype'
      var:	'name', 'modality'

In [ ]:
# now prepare data for the model
from scipy.io import mmwrite
# RNA_count.mtx
mmwrite(data_prefix+'RNA-seq/RNA_count.mtx', mdata['rna'].X.T)
# gene.tsv
mdata['rna'].var.to_csv(data_prefix+'RNA-seq/gene.tsv', header=False)
# barcode.tsv
mdata['rna'].obs.to_csv(data_prefix+'RNA-seq/barcode.tsv', header=False)
# ATAC_count.mtx
mmwrite(data_prefix+'ATAC-seq/ATAC_count.mtx', mdata['atac'].X.T)
# peak.tsv
mdata['atac'].var.to_csv(data_prefix+'ATAC-seq/peak.tsv', header=False)
# barcode.tsv
mdata['atac'].obs.to_csv(data_prefix+'ATAC-seq/barcode.tsv', header=False)

In [ ]:
mdata['atac'].X.shape

(3143, 95677)

In [ ]:
len(mdata['atac'].var)

95677

In [ ]:
len(mdata['atac'].obs)

3143

In [ ]:
mdata['rna'].X.shape

(3143, 15172)

In [ ]:
len(mdata['rna'].var)

15172

In [ ]:
len(mdata['rna'].obs)

3143

# scMM

In [ ]:
!pip install torchnet pyro-ppl

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.5/732.5 kB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 83.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torchnet: filename=torchnet-0.0.4-py3-none-any.whl size=29728 sha256=8b38f759f6785dd989c24d19dfb44369c1dc45beaab1a0409b26683d0378ce8a
  Stored in directory: /root/.cache/pip/wheels/f7/ae/94/9f5edd6871983f30967ad11d60ef434c3d1b007654de4c8065
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408196 sha256=0c99ee491a37cb76c1e788bd6b97f17242d57864695c023273deb5f43cbca2ea
  Stored in directory: /root/.cache/pip/wheels/42/29/49/5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
Successfully built torchnet visdom


In [ ]:
!pip install umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82816 sha256=3070b959c6b95d6a0b627740cf8c9576329fb2d40ab7adc3377e661c4aa4729c
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55622 sha256=4e7b51323359bdb32bb828e54720cc78e6fddff42bb6807814797cd272b9fbb9
  Stored in directory: /root/.cache/pip/wheels/4a/38/5d/f60a40a66a9512b7e5e83517ebc2d1b42d857be97d135f1096
Successfully built umap-learn pynndescent


In [ ]:
!python scMM/src/main.py --experiment 'brain' --model 'rna_atac' --epochs 100 --dataset_path './brain/' --train_val_test_split 'train_val_test_split.csv' --seed 0 --latent_dim 20

2023-07-19 15:52:23.682848: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-19 15:52:24.626217: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
../experiments/brain/2023-07-19T15:52:33.25243675yia24a
Loading  data ...
Original data contains 3143 cells x 15172 peaks
Finished loading takes 0.25 min
Loading  data ...
Original data contains 3143 cells x 95677 peaks
Finished loading takes 0.65 min
RNA-seq shape is (3143, 15172)
ATAC-seq shape is (3143, 95677)
Namespace(experiment='brain', model='rna_atac', obj='m_elbo_naive_warmup', llik_scaling=1.0, batch_size=64, epochs=100, lr=0.0001, latent_dim=20, num_hidden_layers=1, r_hidden_dim=100, p_hidden_dim=20, pre_trained='', learn_prior=False, anal

In [ ]:
!python scMM/src/main.py --experiment 'brain' --model 'rna_atac' --epochs 100 --dataset_path './brain/' --train_val_test_split 'train_val_test_split.csv' --seed 37 --latent_dim 20

2023-07-19 16:02:57.055929: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-19 16:02:58.764115: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
../experiments/brain/2023-07-19T16:03:01.330121b5b7mfmh
Loading  data ...
Original data contains 3143 cells x 15172 peaks
Finished loading takes 0.14 min
Loading  data ...
Original data contains 3143 cells x 95677 peaks
Finished loading takes 0.53 min
RNA-seq shape is (3143, 15172)
ATAC-seq shape is (3143, 95677)
Namespace(experiment='brain', model='rna_atac', obj='m_elbo_naive_warmup', llik_scaling=1.0, batch_size=64, epochs=100, lr=0.0001, latent_dim=20, num_hidden_layers=1, r_hidden_dim=100, p_hidden_dim=20, pre_trained='', learn_prior=False, anal

In [ ]:
!python scMM/src/main.py --experiment 'brain' --model 'rna_atac' --epochs 100 --dataset_path './brain/' --train_val_test_split 'train_val_test_split.csv' --seed 8790 --latent_dim 20

2023-07-19 16:12:52.970395: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-19 16:12:53.884469: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
../experiments/brain/2023-07-19T16:12:55.533731rf6h7d3t
Loading  data ...
Original data contains 3143 cells x 15172 peaks
Finished loading takes 0.17 min
Loading  data ...
Original data contains 3143 cells x 95677 peaks
Finished loading takes 0.52 min
RNA-seq shape is (3143, 15172)
ATAC-seq shape is (3143, 95677)
Namespace(experiment='brain', model='rna_atac', obj='m_elbo_naive_warmup', llik_scaling=1.0, batch_size=64, epochs=100, lr=0.0001, latent_dim=20, num_hidden_layers=1, r_hidden_dim=100, p_hidden_dim=20, pre_trained='', learn_prior=False, anal

## Further steps

1. download `lat_train_mean.csv`, `pred_test_r_r.mtx` and `pred_test_p_p.mtx` for every run.
2. rename them to `scmm_lat_train_mean_rs{seed}.csv`, `scmm_counts_gex_rs{seed}.mtx` and `scmm_counts_atac_rs{seed}.mtx`, respectively.
3. save them into `./experiments/results/other_models/scMM/`